# Train Stable Diffusion with CodeFlare

In [1]:
!pwd

/opt/app-root/src/teddy-demo


In [47]:
!pip install codeflare-sdk
!pip install -Ur 05_dreambooth_finetuning/dreambooth/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 39.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.6
    Uninstalling urllib3-2.0.6:
      Successfully uninstalled urllib3-2.0.6
  Attempting uninstall: charset-normalizer
    Found existing installation: charset-normalizer 3.3.0
    Uninstalling charset-normalizer-3.3.0:
      Successfully uninstalled charset-normalizer-3.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
codeflare 0.1.2.dev0 requires pickle5>=0.0.11, which is not installed.

[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


## Prep Infra

In [4]:
# Import pieces from codeflare-sdk
from codeflare_sdk.cluster.cluster import Cluster, ClusterConfiguration
from codeflare_sdk.cluster.auth import TokenAuthentication

In [5]:
# Create authentication object for user permissions
# IF unused, SDK will automatically check for default kubeconfig, then in-cluster config
# KubeConfigFileAuthentication can also be used to specify kubeconfig path manually
auth = TokenAuthentication(
    token = "sha256~oBoJq1LrYfhkxEFMfROmDKEkvW9I5Ag81ZHCHyo0Eek",
    server = "https://api.aisrhods-dim.t0xq.p1.openshiftapps.com:6443",
    skip_tls=False
)
auth.login()

'Logged into https://api.aisrhods-dim.t0xq.p1.openshiftapps.com:6443'

In [6]:
cluster = Cluster(ClusterConfiguration(
    name='stab-diff-model',
    namespace='default',
    num_workers=2,
    min_cpus=16,
    max_cpus=16,
    min_memory=64,
    max_memory=64,
    num_gpus=1,
    instascale=True, #<---instascale enabled
    machine_types=["m5.xlarge", "g5.4xlarge"],
))

Written to: stab-diff-model.yaml


In [9]:
cluster.up()

In [13]:
# cluster.down()

In [12]:
cluster.details()

                       🚀 CodeFlare Cluster Details 🚀                      
                                                                            
 ╭────────────────────────────────────────────────────────────────────────╮ 
 │   Name                                                                 │ 
 │   stab-diff-model                                        Inactive ❌   │ 
 │                                                                        │ 
 │   URI: ray://stab-diff-model-head-svc.default.svc:10001                │ 
 │                                                                        │ 
 │   Dashboard🔗                                                          │ 
 │                                                                        │ 
 │                       Cluster Resources                                │ 
 │   ╭── Workers ──╮  ╭───────── Worker specs(each) ─────────╮            │ 
 │   │  # Workers  │  │  Memory      CPU         GPU         │            │ 
 │   │             │  │                                      │            │ 
 │   │  2          │  │  64~64       16          1           │            │ 
 │   │             │  │                                      │            │ 
 │   ╰─────────────╯  ╰──────────────────────────────────────╯            │ 
 ╰────────────────────────────────────────────────────────────────────────╯

RayCluster(name='stab-diff-model', status=<CodeFlareClusterStatus.QUEUEING: 4>, workers=2, worker_mem_min=64, worker_mem_max=64, worker_cpu=16, worker_gpu=1, namespace='default', dashboard='Dashboard route not available yet, have you run cluster.up()?')

In [11]:
cluster.status()

╭────────────────────────────────╮
│   🚀 Cluster Queue Status 🚀   │
│ +-----------------+----------+ │
│ | Name            | Status   | │
│ +=================+==========+ │
│ | stab-diff-model | queueing | │
│ |                 |          | │
│ +-----------------+----------+ │
╰────────────────────────────────╯

(<CodeFlareClusterStatus.QUEUEING: 4>, False)

In [ ]:
import codeflare_sdk as cf

In [63]:
cf.cluster.cluster.list_all_clusters('default')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_344/2159714274.py:1 in <module>                                                   │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_344/2159714274.py'                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'list_all_clusters' is not defined

## Step 0: Prep

In [13]:
!pwd

/opt/app-root/src/teddy-demo


In [51]:
# # THIS IS ORIGINAL

# # TODO: If running on multiple nodes, change this path to a shared directory (ex: NFS)
# !export DATA_PREFIX="/opt/app-root/src/teddy-demo"
# !export ORIG_MODEL_NAME="CompVis/stable-diffusion-v1-4"
# !export ORIG_MODEL_HASH="b95be7d6f134c3a9e62ee616f310733567f069ce"
# !export ORIG_MODEL_DIR="$DATA_PREFIX/model-orig"
# !export ORIG_MODEL_PATH="$ORIG_MODEL_DIR/models--${ORIG_MODEL_NAME/\//--}/snapshots/$ORIG_MODEL_HASH"
# !export TUNED_MODEL_DIR="$DATA_PREFIX/model-tuned"
# !export IMAGES_REG_DIR="$DATA_PREFIX/images-reg"
# !export IMAGES_OWN_DIR="$DATA_PREFIX/images-own"
# !export IMAGES_NEW_DIR="$DATA_PREFIX/images-new"
# # TODO: Add more worker nodes and increase NUM_WORKERS for more data-parallelism
# !export NUM_WORKERS=2

# # !mkdir -p 
# !echo $ORIG_MODEL_DIR 
# # $TUNED_MODEL_DIR $IMAGES_REG_DIR $IMAGES_OWN_DIR $IMAGES_NEW_DIR
# !mkdir -p $ORIG_MODEL_DIR $TUNED_MODEL_DIR $IMAGES_REG_DIR $IMAGES_OWN_DIR $IMAGES_NEW_DIR

/model-orig
mkdir: cannot create directory ‘/model-orig’: Permission denied
mkdir: cannot create directory ‘/model-tuned’: Permission denied
mkdir: cannot create directory ‘/images-reg’: Permission denied
mkdir: cannot create directory ‘/images-own’: Permission denied
mkdir: cannot create directory ‘/images-new’: Permission denied


In [14]:
import os

In [15]:
# THIS IS ORIGINAL

# TODO: If running on multiple nodes, change this path to a shared directory (ex: NFS)
DATA_PREFIX="/opt/app-root/src/teddy-demo"

# DATA_PREFIX="/opt/app-root/src/text-to-image-demo/dim-dreambooth"
ORIG_MODEL_NAME="CompVis/stable-diffusion-v1-4"
ORIG_MODEL_HASH="b95be7d6f134c3a9e62ee616f310733567f069ce"
ORIG_MODEL_DIR="model-orig"
ORIG_MODEL_PATH="$ORIG_MODEL_DIR/models--${ORIG_MODEL_NAME/\//--}/snapshots/$ORIG_MODEL_HASH"
TUNED_MODEL_DIR="model-tuned"
IMAGES_REG_DIR="images-reg"
IMAGES_OWN_DIR="images-own"
IMAGES_NEW_DIR="images-new"
# TODO: Add more worker nodes and increase NUM_WORKERS for more data-parallelism
!export NUM_WORKERS=2

os.makedirs(ORIG_MODEL_DIR, exist_ok=True)
os.makedirs(TUNED_MODEL_DIR, exist_ok=True)
os.makedirs(IMAGES_REG_DIR, exist_ok=True)
os.makedirs(IMAGES_OWN_DIR, exist_ok=True)
os.makedirs(IMAGES_NEW_DIR, exist_ok=True)

In [16]:
# !export DATA_PREFIX="/opt/app-root/src/text-to-image-demo/dim-dreambooth"
# !export ORIG_MODEL_DIR="$DATA_PREFIX/model-orig"
# !export ORIG_MODEL_NAME="CompVis/stable-diffusion-v1-4"
# !export ORIG_MODEL_HASH="b95be7d6f134c3a9e62ee616f310733567f069ce"


In [17]:
!echo $DATA_PREFIX
!echo $ORIG_MODEL_NAME
!echo $ORIG_MODEL_HASH

/opt/app-root/src/teddy-demo
CompVis/stable-diffusion-v1-4
b95be7d6f134c3a9e62ee616f310733567f069ce


## Step 1: Download the pre-trained model

In [23]:
!pwd

/opt/app-root/src/teddy-demo


In [44]:
# !python 05_dreambooth_finetuning/dreambooth/cache_model.py --model_dir=$DATA_PREFIX/images-reg --model_name=$ORIG_MODEL_NAME --revision=$ORIG_MODEL_HASH

## Step 2: Supply images of your subject

In [19]:
  # Only uncomment one of the following:

  # Option 1: Use the dog dataset ---------
!export CLASS_NAME="dog"
# !python 05_dreambooth_finetuning/dreambooth/download_example_dataset.py ./images/dog
CLASS_NAME='dog'
  # export INSTANCE_DIR=./images/dog
  # ---------------------------------------

  # Option 2: Use the lego car dataset ----
 # !export CLASS_NAME="car"
 # !export INSTANCE_DIR=./images/lego-car
  # ---------------------------------------

  # Option 3: Use your own images ---------
  # export CLASS_NAME="<class-of-your-subject>"
  # export INSTANCE_DIR="/path/to/images/of/subject"
  # ---------------------------------------

  # Copy own images into IMAGES_OWN_DIR
# !cp -rf $INSTANCE_DIR/* "$IMAGES_OWN_DIR/"

## Step 3: Create the regularization images

In [20]:
!echo $CLASS_NAME

dog


In [21]:
!pip install flags

  Preparing metadata (setup.py) ... done
  Created wheel for flags: filename=flags-0.0.1.2-py3-none-any.whl size=1700 sha256=82dfce04fc09a2a405ad736aa54a99dcb1095c6cc19f548a242032f1afa76f5a
  Stored in directory: /tmp/pip-ephem-wheel-cache-r5jmbmyt/wheels/56/d8/66/3b5a98f9680b2719b25fe1d812d205577bd53fc642bd5902c9
Successfully built flags

[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [22]:
!pip install pyarrow


[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [23]:
!python 05_dreambooth_finetuning/dreambooth/generate.py \
    --model_dir=ORIG_MODEL_PATH \
    --output_dir=IMAGES_REG_DIR \
    --prompts=f"photo of a dog" \
    --num_samples_per_prompt=200 \
    --use_ray_data

^C


## Fine tune the model

In [24]:
# !pip install ray['train'] --upgrade --force-reinstall

In [25]:
UNIQUE_TOKEN = 'unqtkn'
CLASS_NAME = 'dog'

In [26]:
!pip install gputil

  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7394 sha256=d550caca4e70ea6b408ce8b9041bbb4d0015add78b39cbed368da43932fa1f85
  Stored in directory: /tmp/pip-ephem-wheel-cache-g9gs7af5/wheels/0f/d6/bc/81bb079b0a87885354d6c32d323698a718a125cd65955155d8
Successfully built gputil

[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [42]:
# !pip install ray['rllib'] --force-reinstall
!pip install ray['Train']==2.5 #--upgrade --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 100.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: ray
    Found existing installation: ray 2.7.1
    Uninstalling ray-2.7.1:
      Successfully uninstalled ray-2.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
codeflare 0.1.2.dev0 requires pickle5>=0.0.11, which is not installed.

[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [28]:
# !pip install ray['Air'] --force-reinstall

In [29]:
# !pip install xformers --force-reinstall

In [30]:
# !pip install grpcio --upgrade --force-reinstall

In [31]:
# !pip install pyarrow --upgrade --force-reinstall

In [45]:
# !pip install ray --upgrade --force-reinstall

In [46]:
from ray.train import ScalingConfig

ImportError: cannot import name 'ScalingConfig' from 'ray.train' (/opt/app-root/lib64/python3.8/site-packages/ray/train/__init__.py)

In [32]:
!python 05_dreambooth_finetuning/dreambooth/train.py \
    --model_dir=images-reg \
    --output_dir=model-tuned \
    --instance_images_dir=images-own \
    --instance_prompt="photo of unqtkn dog" \
    --class_images_dir=images-reg \
    --class_prompt="photo of a dog" \
    --train_batch_size=2 \
    --lr=5e-6 \
    --num_epochs=4 \
    --max_train_steps=200 \
    --num_workers=2

╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /opt/app-root/src/teddy-demo/05_dreambooth_finetuning/dreambooth/train.py:27 │
│ in <module>                                                                  │
│                                                                              │
│    24                                                                        │
│    25 # LoRA related imports end ##                                          │
│    26 from diffusers.utils.import_utils import is_xformers_available         │
│ ❱  27 from ray.train import ScalingConfig                                    │
│    28 from ray import train                                                  │
│    29 from ray.train.torch import TorchTrainer                               │
│    30 import torch                                                           │
╰──────────────────────────────────────────────────────────────────────────────╯
ImportError: cannot import n

In [ ]:
cluster.details()

In [ ]:
from codeflare_sdk.job.jobs import DDPJobDefinition

In [ ]:
arg_list = [
    "--model_name_or_path", "gpt2",
    "--dataset_name", "wikitext",
    "--dataset_config_name", "wikitext-2-raw-v1",
    "--per_device_train_batch_size", "2",
    "--per_device_eval_batch_size", "2",
    "--do_train",
    "--do_eval",
    "--output_dir", "/tmp/test-clm",
    "--overwrite_output_dir"
]

In [ ]:
jobdef = DDPJobDefinition(
    name="gpttest",
    script="gpt_og.py",
    script_args=arg_list,
    scheduler_args={"requirements": "requirements_gpt.txt"}
)
job = jobdef.submit(cluster)

In [ ]:
job.status()

Retrieve raw log output at anytime with:

In [ ]:
job.logs()

View live updates for status, logs, and other information with:

In [ ]:
cluster.cluster_dashboard_uri()

In [ ]:
job.status()

In [ ]:
cluster.down()

In [ ]:
auth.logout()